## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob

## PD Set Options

In [2]:
pd.set_option('display.max_columns',None)

## Data Read-in

In [8]:
csv_file_list = glob.glob('*.csv')

df_list = []

for csv_file in csv_file_list:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [9]:
df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,"In accordance with local MLS rules, some MLS l...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PAST SALE,NaN,Condo/Co-op,453 Fanshaw K #453,Boca Raton,FL,33434.0,275000.0,2.0,1.5,NaN,835.0,NaN,1980.0,NaN,329.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/453-Fansh...,NaN,NaN,N,Y,26.384826,-80.174459
2,PAST SALE,NaN,Condo/Co-op,618 NW 13th St #110,Boca Raton,FL,33486.0,300000.0,3.0,2.0,NaN,1286.0,NaN,1970.0,NaN,233.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/618-NW-13...,NaN,NaN,N,Y,26.360975,-80.099405
3,PAST SALE,October-31-2023,Condo/Co-op,230 Somerset L,West Palm Beach,FL,33417.0,300000.0,2.0,2.0,Century Village,874.0,NaN,1972.0,NaN,343.0,500.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/230-...,Beaches MLS,RX-10921251,N,Y,26.713297,-80.130859
4,PAST SALE,November-3-2023,Condo/Co-op,450 N Federal Hwy #603,Boynton Beach,FL,33435.0,283500.0,1.0,1.0,Casa Costa Condo,805.0,NaN,2010.0,NaN,352.0,573.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/450-N-...,Beaches MLS,RX-10917527,N,Y,26.530064,-80.057953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,PAST SALE,NaN,Condo/Co-op,709 Lori Dr #314,Palm Springs,FL,33461.0,155000.0,1.0,1.0,NaN,696.0,NaN,1979.0,NaN,223.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Palm-Springs/709-Lor...,NaN,NaN,N,Y,26.646112,-80.093287
216,PAST SALE,NaN,Condo/Co-op,29 Waterford B,Delray Beach,FL,33446.0,150000.0,2.0,2.0,NaN,883.0,NaN,1973.0,NaN,170.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/29-Wate...,NaN,NaN,N,Y,26.448901,-80.151942
217,PAST SALE,NaN,Condo/Co-op,1251 NW 13th St Unit 433C,Boca Raton,FL,33486.0,165000.0,1.0,1.0,NaN,729.0,NaN,1973.0,NaN,226.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1251-NW-1...,NaN,NaN,N,Y,26.361855,-80.111931
218,PAST SALE,NaN,Condo/Co-op,417 Bennington Ln,Lake Worth,FL,33467.0,175000.0,2.0,2.0,NaN,941.0,NaN,1983.0,NaN,186.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/417-Benni...,NaN,NaN,N,Y,26.624736,-80.160075


In [10]:
PBC_gf = gpd.read_file('Palm_Beach_County_Boundary.geojson')

## Data Clean

In [11]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [12]:
df = df.dropna(subset=['SOLD DATE'])

In [13]:
# Define list of desired months (excluding current month)
desired_months = ['November']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [14]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    581
Name: count, dtype: int64
-------
$/SQUARE FEET
False    578
True       3
Name: count, dtype: int64
-------
YEAR BUILT
False    581
Name: count, dtype: int64
-------


In [15]:
sorted_df = df_filtered.sort_values(by='$/SQUARE FEET', ascending=True)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Lake-Worth/2566-Garden-Dr-S-33461/unit-205/home/42285967


In [16]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [17]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [18]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
382,PAST SALE,November-22-2023,Condo/Co-op,2811 Garden Dr S #105,Lake Worth,FL,33461.0,55000.0,1.0,1.5,Lake Clarke Gardens Condo 25,744.0,NaN,1971.0,NaN,74.0,737.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2811-Gard...,Beaches MLS,RX-10914956,N,Y,26.633652,-80.085226
392,PAST SALE,November-3-2023,Condo/Co-op,136 Sheffield F,West Palm Beach,FL,33417.0,68000.0,1.0,1.0,Century Village,570.0,NaN,1971.0,NaN,119.0,409.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/136-...,Beaches MLS,RX-10891556,N,Y,26.719117,-80.126059
404,PAST SALE,November-20-2023,Condo/Co-op,3360 Lake Osborne Dr #206,Lake Worth,FL,33461.0,70000.0,1.0,1.5,Murry Hills Apt Bldg Condo,734.0,NaN,1979.0,NaN,95.0,287.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth-Beach/336...,Beaches MLS,RX-10858446,N,Y,26.603926,-80.073442
394,PAST SALE,November-2-2023,Condo/Co-op,280 Windsor M,West Palm Beach,FL,33417.0,70000.0,1.0,1.0,Century Village,585.0,NaN,1972.0,NaN,120.0,410.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/280-...,Beaches MLS,RX-10886223,N,Y,26.711138,-80.135877
398,PAST SALE,November-9-2023,Condo/Co-op,166 Salisbury G #166,West Palm Beach,FL,33417.0,72000.0,1.0,1.0,SALISBURY CONDOS,570.0,NaN,1969.0,NaN,126.0,445.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/166-...,MARMLS,A11353819,N,Y,26.716733,-80.121228
391,PAST SALE,November-15-2023,Condo/Co-op,3500 Springdale Blvd #115,Palm Springs,FL,33461.0,72500.0,1.0,1.5,REGAL PALMS REGENT CONDO,779.0,NaN,1980.0,NaN,93.0,350.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Springs/3500-Sp...,MARMLS,A11413329,N,Y,26.646890,-80.103729
363,PAST SALE,November-24-2023,Condo/Co-op,1956 Bridgewood Dr,Boca Raton,FL,33434.0,75000.0,2.0,2.0,Bridgewood Midrise,1050.0,NaN,1974.0,NaN,71.0,729.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1956-Brid...,Beaches MLS,RX-10926386,N,Y,26.373894,-80.164420
381,PAST SALE,November-9-2023,Condo/Co-op,158 Easthampton Unit G,West Palm Beach,FL,33417.0,77000.0,1.0,1.5,EASTHAMPTON CONDO,702.0,NaN,1969.0,NaN,110.0,445.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/158-...,MARMLS,A11408474,N,Y,26.716714,-80.119845
393,PAST SALE,November-28-2023,Condo/Co-op,2566 S Garden Dr #205,Lake Worth,FL,33461.0,79000.0,2.0,1.0,Lake Clarke Gardens Condo 19,894.0,NaN,1969.0,NaN,88.0,625.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2566-Gard...,Beaches MLS,RX-10887550,N,Y,26.632465,-80.081298
401,PAST SALE,November-8-2023,Condo/Co-op,155 Andover F #155,West Palm Beach,FL,33417.0,80000.0,1.0,1.0,ANDOVER CONDOS,532.0,NaN,1971.0,NaN,150.0,450.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/155-...,MARMLS,A11346109,N,Y,26.710947,-80.128411


In [20]:
print(df_filtered['URL'].iloc[392])

https://www.redfin.com/FL/West-Palm-Beach/136-Sheffield-F-33417/home/42145053


In [95]:
# # Correct the prices, if needed
# df_filtered.at[43,'$/SQUARE FEET']=(275000/705)

In [21]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [97]:
# # # Corrections, if needed
# df_filtered.at[245,'$/SQUARE FEET']=(210000/480)
# df_filtered.at[383,'$/SQUARE FEET']=(225000/460)
# df_filtered.at[673,'$/SQUARE FEET']=(550000/960)
# df_filtered.at[777,'$/SQUARE FEET']=(275000/697)

In [22]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
363,75000.0,1956 Bridgewood Dr,Boca Raton,71.0
382,55000.0,2811 Garden Dr S #105,Lake Worth,74.0
393,79000.0,2566 S Garden Dr #205,Lake Worth,88.0
385,80000.0,2647 N Garden Dr #104,Lake Worth,89.0
359,95000.0,14721 Bonaire Blvd #208,Delray Beach,93.0
391,72500.0,3500 Springdale Blvd #115,Palm Springs,93.0
404,70000.0,3360 Lake Osborne Dr #206,Lake Worth,95.0
368,88000.0,73 Canterbury Unit C,West Palm Beach,110.0
381,77000.0,158 Easthampton Unit G,West Palm Beach,110.0
576,155000.0,34 Southport Ln Unit E,Boynton Beach,112.0


In [23]:
print(df_filtered.URL.iloc[363])

https://www.redfin.com/FL/Boca-Raton/1956-Bridgewood-Dr-33434/home/187918847


In [100]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [24]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [25]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [26]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [27]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [28]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [29]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"November 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(PBC_gf,tooltip='Palm Beach County',name='Palm Beach County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [30]:
m.save('index.html')

## Summary Info

In [31]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [32]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [33]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [43]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth/2811-Garden-Dr-S-33461/unit-105/home/42287192


In [40]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
La Clara, 200 Arkona Ct West Palm Beach | Price $18,750,000 | $2,338 psf | Year built: 2023
Least Expensive
Lake Clarke Gardens Condo 25, 2811 Garden Dr S #105 Lake Worth | Price $55,000 | $74 psf | Year built: 1971
Highest Price Per Square Foot
Seaglass Condominium, 1500 Beach Rd #403 Tequesta | Price $12,500,000 | $2,399 psf | Year built: 2022
Lowest Price Per Square Foot
Bridgewood Midrise, 1956 Bridgewood Dr Boca Raton | Price $75,000 | $71 psf | Year built: 1974
Newest
La Clara, 200 Arkona Ct West Palm Beach | Price $18,750,000 | $2,338 psf | Year built: 2023
Oldest
Gulfstream Condo, 31 S Golfview Rd #13 Lake Worth Beach | Price $235,000 | $324 psf | Year built: 1948


In [36]:
df_filtered['YEAR BUILT'].sort_values()

179    1948.0
492    1956.0
521    1963.0
288    1964.0
274    1964.0
        ...  
469    2023.0
467    2023.0
463    2023.0
461    2023.0
530    2023.0
Name: YEAR BUILT, Length: 581, dtype: float64

## Time on Market Calculator

In [30]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth/2855-Garden-Dr-S-33461/unit-312/home/42291183


In [42]:
from datetime import datetime, timedelta

date1 = datetime(2021, 10, 12) ## List (Earlier) date
date2 = datetime(2023, 11, 3) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

752


## Map URL Snagger

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_month_ending_nov_2023


## Get Summary Data

In [37]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 581
--------
Total sale price: $299,104,047
--------
Median sale price: $260,000
--------
Max sale price: $18,750,000
--------
Min sale price: $55,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,399
--------
Min price per square foot: $71
--------
Median price per square foot: $232
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1948.0
----------
Average building age: 1983.5180722891566
